In [ ]:
#Date: 9/26/2022
#Author: Talia Kurtz

#What is the purpose of this code?
#This code further drops unwanted variables from the ARM reduced winter files for each year and then NaNs data that was denoted in the quality control report as being incorrect or suspect. Once the
#incorrect and or suspect data is replaced as a NaN then the QC variables are run through each vairable and any value that deviates from 0 is then NaNed out. The individual variables are saved as seperate
#files to be globbed together into one large file.

In [26]:
#Imports
from datetime import datetime

import cfgrib
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
import metpy.calc as mpcalc
from metpy.units import units
import scipy.ndimage as ndimage
import pandas as pd
import itertools

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

#Open the ARM data file that I want 
ARM_PATH = '/Users/research/thesis_code/ARM_data_winter_months/'
dg = xr.open_dataset(ARM_PATH + 'ARM_2012_reduced.nc')

In [27]:
#dx = dg.drop(['qc_logger_volt', 'pws_mean_vis_10min','qc_pws_mean_vis_10min', 'pws_pw_code_15min', 'qc_pws_pw_code_15min','pws_pw_code_1hr', 'qc_pws_pw_code_1hr'])
dx = dg.drop(['qc_logger_volt', 'pwd_mean_vis_10min','qc_pwd_mean_vis_10min', 'pwd_pw_code_15min', 'qc_pwd_pw_code_15min','pwd_pw_code_1hr', 'qc_pwd_pw_code_1hr'])
dx

<xarray.Dataset>
Dimensions:                       (time: 5112)
Coordinates:
  * time                          (time) datetime64[ns] 2012-01-01 ... 2012-1...
Data variables: (12/28)
    qc_time                       (time) float64 ...
    atmos_pressure                (time) float32 ...
    qc_atmos_pressure             (time) float64 ...
    temp_mean                     (time) float32 ...
    qc_temp_mean                  (time) float64 ...
    temp_std                      (time) float32 ...
    ...                            ...
    qc_pwd_precip_rate_mean_1min  (time) float64 ...
    pwd_cumul_snow                (time) float32 ...
    qc_pwd_cumul_snow             (time) float64 ...
    lat                           (time) float32 ...
    lon                           (time) float32 ...
    alt                           (time) float32 ...

In [28]:
start =datetime.strptime("2012-01-01 00", "%Y-%m-%d %H")
end = datetime.strptime("2012-12-31 23", "%Y-%m-%d %H")
date_generated = pd.date_range(start, end, freq='1H')
list_1 = date_generated.strftime("%Y-%m-%d %H")
print(list_1)

Index(['2012-01-01 00', '2012-01-01 01', '2012-01-01 02', '2012-01-01 03',
       '2012-01-01 04', '2012-01-01 05', '2012-01-01 06', '2012-01-01 07',
       '2012-01-01 08', '2012-01-01 09',
       ...
       '2012-12-31 14', '2012-12-31 15', '2012-12-31 16', '2012-12-31 17',
       '2012-12-31 18', '2012-12-31 19', '2012-12-31 20', '2012-12-31 21',
       '2012-12-31 22', '2012-12-31 23'],
      dtype='object', length=8784)


In [29]:
bad_data_gen=list_1

In [30]:
for val in time:
    dx['temp_mean'].loc[val]= np.nan
    dx['rh_mean'].loc[val]= np.nan
    dx['atmos_pressure'].loc[val]= np.nan
    dx['wspd_arith_mean'].loc[val]= np.nan
    dx['wdir_vec_mean'].loc[val]= np.nan
    dx['pwd_mean_vis_1min'].loc[val]= np.nan
    dx['pwd_precip_rate_mean_1min'].loc[val]= np.nan
    dx['pwd_cumul_snow'].loc[val]= np.nan

In [31]:
temp = dx['temp_mean']
temp_mask = temp.where(dx['qc_temp_mean'] ==0)
###############################################################################################
pressure = dx['atmos_pressure']
pressure_mask = pressure.where(dx['qc_atmos_pressure'] ==0)
###############################################################################################
rh = dx['rh_mean']
rh_mask = rh.where(dx['qc_rh_mean'] ==0)
###############################################################################################
wspd = dx['wspd_arith_mean']
wspd_mask = wspd.where(dx['qc_wspd_arith_mean'] ==0)
###############################################################################################
wdir = dx['wdir_vec_mean']
wdir_mask = wdir.where(dx['qc_wdir_vec_mean'] ==0)
###############################################################################################
vis = dx['pwd_mean_vis_1min']
vis_mask = vis.where(dx['qc_pwd_mean_vis_1min'] ==0)
###############################################################################################
precip_rate = dx['pwd_precip_rate_mean_1min']
precip_rate_mask = precip_rate.where(dx['qc_pwd_precip_rate_mean_1min'] ==0)
###############################################################################################
snow = dx['pwd_cumul_snow']
snow_mask = snow.where(dx['qc_pwd_cumul_snow'] ==0)

In [32]:
temp_mask.to_netcdf('/Users/research/thesis_code/ARM_data_winter_months/ARM_2012_temp_mask.nc')
pressure_mask.to_netcdf('/Users/research/thesis_code/ARM_data_winter_months/ARM_2012_pressure_mask.nc')
rh_mask.to_netcdf('/Users/research/thesis_code/ARM_data_winter_months/ARM_2012_rhmask.nc')
wspd_mask.to_netcdf('/Users/research/thesis_code/ARM_data_winter_months/ARM_2012_wspdmask.nc')
wdir_mask.to_netcdf('/Users/research/thesis_code/ARM_data_winter_months/ARM_2012_wdirmask.nc')
vis_mask.to_netcdf('/Users/research/thesis_code/ARM_data_winter_months/ARM_2012_vismask.nc')
precip_rate_mask.to_netcdf('/Users/research/thesis_code/ARM_data_winter_months/ARM_2012_precipratemask.nc')
snow_mask.to_netcdf('/Users/research/thesis_code/ARM_data_winter_months/ARM_2012_cumul_snow_mask.nc')